In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import sys
sys.path.append('/DATA/publish/mocet/analysis/scripts')
from utils.base import get_project_directory, get_configs

project_dir = get_project_directory()
configs = get_configs()

x_direction_range = np.linspace(-np.deg2rad(5), np.deg2rad(5), 9)
z_direction_range = np.linspace(-np.deg2rad(5), np.deg2rad(5), 9)
orientation_range = np.arange(-90, 91, 5)
depth_range = [0.14,0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25]
total_count = len(x_direction_range)*len(z_direction_range)*len(depth_range)*len(orientation_range)*12
print(total_count, total_count/12)

basis_log_fname = f'../../data/simulation/basis_log.csv'
basis_log = pd.read_csv(basis_log_fname)
basis_pupill_data = basis_log[["center_x","center_y","confidence"]]
basis_pupill_data = basis_pupill_data.dropna()
basis_pupill_data = basis_pupill_data.to_numpy()

basis = []
basis_params = []
idx = 0
for x, x_d in enumerate(x_direction_range):
    for z, z_d in enumerate(z_direction_range):
        for depth in depth_range:
            for d, deg in enumerate(orientation_range):
                tmp_basis = np.zeros((12,2))
                for t in range(12):
                    tmp_basis[t,:] = basis_pupill_data[idx, :2]
                    idx += 1
                if np.any(tmp_basis == -1):
                    pass
                else:
                    basis.append(tmp_basis)
                    basis_params.append([x_d, z_d, depth, deg])

basis = np.array(basis)
basis_params = np.array(basis_params)

valid_data = pickle.load(open('../../data/valid_data_list.pkl', 'rb'))

subjects = []
subjects_runs = {}
for key in list(valid_data.keys()):
    subjects.append(key[0])
    if not subjects_runs.keys().__contains__(key[0]):
        subjects_runs[key[0]] = 0
    subjects_runs[key[0]] += 1
subjects = list(set(subjects))
subjects.sort()

In [ ]:
import os
import mocet

calibration_onsets = configs['calibration_onsets']
calibration_points = configs['calibration_points']
interval = configs['interval']
task_duration = configs['task_duration']
task = configs['task']

calibration_offset_start = configs['calibration_offset_start']
calibration_offset_end = configs['calibration_offset_end']
calibration_threshold = configs['calibration_threshold']
px_per_deg = configs['px_per_deg']
avg_pupil_diameter_mm = configs['avg_pupil_diameter_mm']
calibration_coordinates = configs['calibration_coordinates']
calibration_order = configs['calibration_order']

validation_errors = []
pupil_displacement = []
pupil_displacement_model = []
t_cal = 0

basis_data = {}
for key in valid_data.keys():
    subject, session, task, run = key
    motion_param_labels = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
    root = f'{project_dir}/data/eyetracking/{subject}/{session}'
    log_fname = f'{root}/{subject}_{session}_{task}_{run}_recording-eyetracking_physio_log.csv'
    data_fname = f'{root}/{subject}_{session}_{task}_{run}_recording-eyetracking_physio_dat.txt'
    confounds_fname = f'{root}/{subject}_{session}_{task}_{run}_desc-confounds_timeseries.tsv'
    history_fname = f'{root}/{subject}_{session}_{task}_{run}_recording-eyetracking_physio_his.txt'
    start, _, _ = mocet.utils.get_viewpoint_history(history_fname)

    pupil_data, pupil_timestamps, pupil_confidence, pupil_diameter = mocet.utils.clean_viewpoint_data(log_fname,
                                                                                                   data_fname,
                                                                                                   start=start,
                                                                                                   duration=task_duration)
    offset = calibration_onsets[t_cal]
    calibration_pupils = []
    for i in np.arange(calibration_points[t_cal]):
        start = (offset + i) * interval + calibration_offset_start
        end = (offset + i + 1) * interval + calibration_offset_end
        log_effective = np.logical_and(pupil_timestamps >= start * 1000, pupil_timestamps < end * 1000)
        calibration_pupils.append([np.nanmean(pupil_data[log_effective, 0]),
                                   np.nanmean(pupil_data[log_effective, 1])])
    calibration_pupils = np.array(calibration_pupils).reshape(2, 12, 2).mean(axis=0)

    basis_similarity = []
    for i in range(len(basis)):
        basis_similarity.append(np.mean(np.sqrt(np.sum((basis[i] - calibration_pupils) ** 2, axis=-1))))
    basis_idx = np.argmin(basis_similarity)
    basis_data[key] = basis_idx

with open('../../data/simulation/basis_data.pkl', 'wb') as f:
    pickle.dump(basis_data, f)

In [ ]:
import os
import mocet

basis_data = pickle.load(open('../../data/simulation/basis_data.pkl', 'rb'))

def generate_eyetracking_simulation(subject, session, task, run, magnification = 1.0):
    output_dir = f'{project_dir}/data/simulation/{magnification}/{subject}/{session}'
    output_fname = f'{output_dir}/{subject}_{session}_{task}_{run}_simulation-eyetracking.csv'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    if not os.path.exists(f'{output_fname}'):
        motion_param_labels = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
        root = f'{project_dir}/data/eyetracking/{subject}/{session}'
        confounds_fname = f'{root}/{subject}_{session}_{task}_{run}_desc-confounds_timeseries.tsv'

        basis_idx = basis_data[(subject, session, task, run)]

        fmriprep_confounds = pd.read_csv(confounds_fname, delimiter='\t')
        motion_params = fmriprep_confounds[motion_param_labels]
        motion_params = np.nan_to_num(motion_params) * magnification
        motion_params = motion_params - motion_params[0, :]

        _, pupil_coordinates = mocet.simulation.generate(motion_params, basis_params[basis_idx],
                                     render=True, render_resolution = (128, 96),detect_pupil=True)
        pupil_coordinates.to_csv(f'{output_fname}', index=False)

In [ ]:
import time
from multiprocessing import Pool

magnifications = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4,
                  2.6, 2.8, 3.0, 3.2, 3.4, 3.6]
for m, magnification in enumerate(magnifications):
    time_sta = time.time()
    n_processes = 60
    with Pool(n_processes) as pool:
        pool.starmap(generate_eyetracking_simulation, [key + (magnification,) for key in valid_data.keys()])
    print(f'magnification:{magnification} Elapsed time {(time.time() - time_sta):3.3f}')